# ivoirshop

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Liste des URLs à traiter
urls = ["https://www.ivoirshop.ci/", ]

# Liste pour stocker les DataFrames de chaque site
all_dfs = []

# Boucle à travers chaque URL
for url in urls:
    # Étape 1 : Envoyer une requête HTTP à la page web et récupérer le contenu HTML
    response = requests.get(url)
    html_content = response.text

    # Étape 2 : Utiliser BeautifulSoup pour parser le contenu HTML
    soup = BeautifulSoup(html_content, "html.parser")

    # Étape 3 : Extraire les informations des produits
    products = []
    for product_tag in soup.find_all("li", class_="product"):
        product_info = {}

        # Extraire le lien du produit
        product_info["lien_produit"] = product_tag.find("a")["href"]

        # Extraire le titre du produit
        product_info["titre_produit"] = product_tag.find("h2", class_="woo-loop-product__title").text.strip()

        # Extraire le prix du produit
        price_tag = product_tag.find("span", class_="price")
        product_info["prix_produit"] = price_tag.find("ins").text.strip() if price_tag and price_tag.find("ins") else None

        # Ajouter les informations du produit à la liste
        products.append(product_info)

    # Étape 4 : Créer un DataFrame à partir de la liste de produits
    df = pd.DataFrame(products)

    # Ajouter le DataFrame à la liste globale
    all_dfs.append(df)
df

,lien_produit,titre_produit,prix_produit
0,https://www.ivoirshop.ci/produit/robe-soiree-m...,Robe Sexy/Soirée Moulante à Paillettes Manches...,8.500 FCFA
1,https://www.ivoirshop.ci/produit/tecno-camon-2...,Tecno CAMON 20 – 6.67″- 8 Go – 256Go – 64Mpx –...,105.000 FCFA
2,https://www.ivoirshop.ci/produit/casquette-cot...,Casquette Côte D’Ivoire CAN 2024,6.000 FCFA
3,https://www.ivoirshop.ci/produit/vuvuzela-can2...,Vuvuzela CAN2024 – Couleur variable,None
4,https://www.ivoirshop.ci/produit/perruque-fans...,Perruque Fans Cheveux Explosion Cote d’Ivoire ...,None
...,...,...,...
185,https://www.ivoirshop.ci/produit/hoverboard-mp...,"Hoverboard Mp3 Music – Bluetooth – Lumineux 6,5”",94.500 FCFA
186,https://www.ivoirshop.ci/produit/boitier-smart...,"Boîtier Smart TV V88 Mini, Android 12, WIFI 2....",32.500 FCFA
187,https://www.ivoirshop.ci/produit/kit-de-protec...,"Kit de protection genouillère, coudière pour e...",6.000 FCFA
188,https://www.ivoirshop.ci/produit/kit-de-protec...,"Kit de protection (casque, genouillère, coudiè...",12.500 FCFA


In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_product_info(url):
    try:
        response = requests.get(url, timeout=500)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {url} : {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    # Extracting product information
    title_tag = soup.find('h1', class_='wt-text-body-03')
    title = title_tag.text.strip() if title_tag else None

    price_tag = soup.find('p', class_='price')
    promo_price_tag = price_tag.find('ins')
    promo_price = promo_price_tag.text.strip() if promo_price_tag else None

    regular_price_tag = price_tag.find('del')
    regular_price = regular_price_tag.text.strip() if regular_price_tag else None

    stock_tag = soup.find('p', class_='stock in-stock')
    stock = stock_tag.text.strip() if stock_tag else None

    description_tag = soup.find('h1', class_='wt-text-body-03')
    description = description_tag.text.strip() if description_tag else None

    # Create a DataFrame with the extracted information
    df_product = pd.DataFrame({
        'Title': [title],
        'promo_price': [promo_price],
        'regular_price': [regular_price],
        'Stock': [stock],
        'Description': [description]
    })

    return df_product

# List of URLs for individual product pages
urls = list(df["lien_produit"])
# Initialize an empty DataFrame to store the results
combined_df = pd.DataFrame()

# Scrape product information for each URL and concatenate the results
for url in urls:
    df_product = scrape_product_info(url)
    combined_df = pd.concat([combined_df, df_product], ignore_index=True)

# Display the combined DataFrame

combined_df


DataFrame pour les produits:



,Title,Prix Promo,Prix Réel,Stock,Description
0,Robe à manches courtes pour vos soirées. Recom...,8.500 FCFA,15.000 FCFA,Status:4 en stock,Robe à manches courtes pour vos soirées. Recom...
1,None,105.000 FCFA,200.000 FCFA,Status:1 en stock,None
2,None,6.000 FCFA,10.000 FCFA,Status:In stock,None
3,None,None,None,None,None
4,None,None,None,None,None
...,...,...,...,...,...
185,None,94.500 FCFA,170.000 FCFA,None,None
186,None,32.500 FCFA,55.000 FCFA,Status:In stock,None
187,None,6.000 FCFA,8.000 FCFA,None,None
188,None,12.500 FCFA,15.000 FCFA,None,None


# kevajo

In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_product_info(product_div):
    title_tag = product_div.find('h3', class_='wd-entities-title')
    title = title_tag.text.strip() if title_tag else None

    price_tag = product_div.find('span', class_='price')
    price = price_tag.text.strip() if price_tag else None

    image_tag = product_div.find('img', class_='attachment-600x498')
    image_url = image_tag['nitro-lazy-src'] if image_tag and 'nitro-lazy-src' in image_tag.attrs else None

    label_tag = product_div.find('span', class_='awl-inner-text')
    label = label_tag.text.strip() if label_tag else None

    product_url = product_div.find('a', class_='product-image-link')['href']

    return {
        'Title': title,
        'Price': price,
        'Image URL': image_url,
        'Label': label,
        'URL': product_url
    }

def scrape_page(url):
    try:
        response = requests.get(url, timeout=500)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {url} : {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    product_divs = soup.find_all('div', class_='wd-product')
    
    products_data = []
    for product_div in product_divs:
        product_info = scrape_product_info(product_div)
        products_data.append(product_info)

    return pd.DataFrame(products_data)

# URL de la page
urls = [
    "https://kevajo.com/"]



# Scrape product information for each URL and concatenate the results
for url in urls:
    df_product = scrape_page(url)
    combined_df = pd.concat([ df_product], ignore_index=True)

# Display the combined DataFrame

combined_df

,Title,Price,Image URL,Label,URL
0,Rouge à Lèvres Liquide Mat Et Crayon à Lèvres ...,"2,000 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,Boutique officielle®,https://kevajo.com/boutique/beaute-et-hygiene/...
1,SLIM EXTREME 4D – Sérum Surconcentré Réducteur...,"9,500 CFA 7,000 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,Boutique officielle®,https://kevajo.com/boutique/eveline-cosmetics/...
2,White Prestige 4D – Crème Blanchissante Pour l...,"5,000 CFA 3,500 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,Boutique officielle®,https://kevajo.com/boutique/eveline-cosmetics/...
3,LUXURY EXPERT 24K GOLD – Lait Corporel Nourris...,"6,000 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,Boutique officielle®,https://kevajo.com/boutique/eveline-cosmetics/...
4,HYALURON CLINIC B5 – Crème Pour Le Contour Des...,"3,500 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,Boutique officielle®,https://kevajo.com/boutique/eveline-cosmetics/...
...,...,...,...,...,...
109,Lush Cherry Eau De Parfum 80ml Rouge Femme,"15,000 CFA 11,000 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,None,https://kevajo.com/boutique/beaute-et-hygiene/...
110,Fiero Black Original Eau De Parfum Pour Homme,"15,000 CFA 10,500 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,None,https://kevajo.com/boutique/beaute-et-hygiene/...
111,Orient Blanc Homme 100ml,"15,000 CFA 10,500 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,None,https://kevajo.com/boutique/beaute-et-hygiene/...
112,Scandant Eau De Parfum 100ML,"15,000 CFA 11,000 CFA",https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,None,https://kevajo.com/boutique/beaute-et-hygiene/...


In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_product_info(product_div):
    title_tag = product_div.find('h3', class_='wd-entities-title')
    title = title_tag.text.strip() if title_tag else None

    price_tag = product_div.find('span', class_='price')
    price = price_tag.text.strip() if price_tag else None

    promo_tag = product_div.find('span', class_='woocommerce-Price-amount amount')
    promo = promo_tag.text.strip() if promo_tag else None

    real_price_tag = product_div.find('ins', class_='woocommerce-Price-amount amount')
    real_price = real_price_tag.text.strip() if real_price_tag else None

    image_tag = product_div.find('img', class_='attachment-600x498')
    image_url = image_tag['nitro-lazy-src'] if image_tag and 'nitro-lazy-src' in image_tag.attrs else None

    label_tag = product_div.find('span', class_='awl-inner-text')
    label = label_tag.text.strip() if label_tag else None

    product_url = product_div.find('a', class_='product-image-link')['href']

    return {
        'Title': title,
        'Price': price,
        'Promo': promo,
        'Real Price': real_price,
        'Image URL': image_url,
        'Label': label,
        'URL': product_url
    }

def scrape_page(url):
    try:
        response = requests.get(url, timeout=500)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {url} : {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    product_divs = soup.find_all('div', class_='wd-product')
    
    products_data = []
    for product_div in product_divs:
        product_info = scrape_product_info(product_div)
        products_data.append(product_info)

    return pd.DataFrame(products_data)

# URL de la page
urls = [
    "https://kevajo.com/"]

# Scrape product information for each URL and concatenate the results
for url in urls:
    df_product = scrape_page(url)
    combined_df = pd.concat([ df_product], ignore_index=True)
combined_df




In [28]:
combined_df

,Title,Price,Promo,Real Price,Image URL,Label,URL
0,Rouge à Lèvres Liquide Mat Et Crayon à Lèvres ...,"2,000 CFA","2,000 CFA",None,https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,Boutique officielle®,https://kevajo.com/boutique/beaute-et-hygiene/...
1,SLIM EXTREME 4D – Sérum Surconcentré Réducteur...,"9,500 CFA 7,000 CFA","9,500 CFA",None,https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,Boutique officielle®,https://kevajo.com/boutique/eveline-cosmetics/...
2,White Prestige 4D – Crème Blanchissante Pour l...,"5,000 CFA 3,500 CFA","5,000 CFA",None,https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,Boutique officielle®,https://kevajo.com/boutique/eveline-cosmetics/...
3,LUXURY EXPERT 24K GOLD – Lait Corporel Nourris...,"6,000 CFA","6,000 CFA",None,https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,Boutique officielle®,https://kevajo.com/boutique/eveline-cosmetics/...
4,HYALURON CLINIC B5 – Crème Pour Le Contour Des...,"3,500 CFA","3,500 CFA",None,https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,Boutique officielle®,https://kevajo.com/boutique/eveline-cosmetics/...
...,...,...,...,...,...,...,...
109,Pegasus Original Eau De Parfum Pour Homme -100ml,"35,000 CFA 11,000 CFA","35,000 CFA",None,https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,None,https://kevajo.com/boutique/beaute-et-hygiene/...
110,LAUNO The One And Only Eau De Parfum 100ml Int...,"35,000 CFA 15,000 CFA","35,000 CFA",None,https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,None,https://kevajo.com/boutique/beaute-et-hygiene/...
111,Oniro Atom Eau De Parfum – 100ml,"15,000 CFA 11,000 CFA","15,000 CFA",None,https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,None,https://kevajo.com/boutique/beaute-et-hygiene/...
112,Lush Cherry Eau De Parfum 80ml Rouge Femme,"15,000 CFA 11,000 CFA","15,000 CFA",None,https://cdn-jgmnn.nitrocdn.com/IeLUGrClUJaCmTO...,None,https://kevajo.com/boutique/beaute-et-hygiene/...


In [38]:
combined_df['URL'][0]

'https://kevajo.com/boutique/beaute-et-hygiene/maquillages/maquillage-levres/rouge-a-levres-liquide-mat-et-crayon-a-levres-inclus/'

In [52]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_text(element, tag_name=None):
    if element and tag_name:
        tag = element.find(tag_name)
        return tag.text.strip() if tag else ""
    return ""

def clean_text(text):
    return text.replace('\r\n', '').replace('\xa0', '')

def scrape_kevajo_page(page_urls):
    product_data_list = []

    for page_url in page_urls:
        try:
            response = requests.get(page_url, timeout=500)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Erreur de connexion à {page_url} : {e}")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')

        # Extraction des éléments de la page
        breadcrumbs = [a.text.strip() for a in soup.select('.woocommerce-breadcrumb a')]
        product_title = extract_text(soup.find('h1', class_='product_title'))
        
        # Extraction du prix promo
        price_ins = None
        price_ins_tag = soup.find('ins', class_='woocommerce-Price-amount')
        if price_ins_tag:
            price_ins = price_ins_tag.find('bdi').text.strip()

        # Extraction du prix réel
        price_del = None
        price_del_tag = soup.find('del', class_='woocommerce-Price-amount')
        if price_del_tag:
            price_del = price_del_tag.find('bdi').text.strip()

        description = extract_text(soup.find('div', class_='woocommerce-product-details__short-description'))

        attributes = {}
        attribute_rows = soup.select('.woocommerce-product-attributes tr')
        for row in attribute_rows:
            label = row.find('th').text.strip()
            value = row.find('td').text.strip()
            attributes[label] = value

        data = {
            'Breadcrumbs': breadcrumbs,
            'ProductTitle': product_title,
            'PricePromo': price_ins,
            'PriceReel': price_del,
            'Description': description,
            'Attributes': attributes,
        }

        product_data_list.append(data)

    return product_data_list

# Liste des URLs de pages produits
page_urls = list(combined_df['URL'])

# Scrape des détails de chaque page
product_data_list = scrape_kevajo_page(page_urls)

# Création d'un DataFrame à partir des données
df = pd.DataFrame(product_data_list)

# Affichage du DataFrame
df

,Breadcrumbs,ProductTitle,PricePromo,PriceReel,Description,Attributes
0,"[Accueil, Tous les produits, Eveline Cosmetics...",,None,None,,"{'Poids': '0.2 kg', 'Marque': 'Eveline Cosmeti..."
1,"[Accueil, Tous les produits, Eveline Cosmetics...",,None,None,,"{'Poids': '0.3 kg', 'Marque': 'Eveline Cosmeti..."
2,"[Accueil, Tous les produits, Eveline Cosmetics...",,None,None,,"{'Poids': '0.1 kg', 'Marque': 'Eveline Cosmeti..."
3,"[Accueil, Tous les produits, Eveline Cosmetics...",,None,None,,"{'Poids': '0.425 kg', 'Marque': 'Eveline Cosme..."
4,"[Accueil, Tous les produits, Eveline Cosmetics...",,None,None,,"{'Poids': '0.08 kg', 'Marque': 'Eveline Cosmet..."
...,...,...,...,...,...,...
109,"[Accueil, Tous les produits, Beauté et hygiène...",,None,None,,{'Poids': '0.3 kg'}
110,"[Accueil, Tous les produits, Beauté et hygiène...",,None,None,,{'Poids': '0.3 kg'}
111,"[Accueil, Tous les produits, Beauté et hygiène...",,None,None,,{'Poids': '0.3 kg'}
112,"[Accueil, Tous les produits, Beauté et hygiène...",,None,None,,{'Poids': '0.3 kg'}


In [59]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_text(element, tag_name=None):
    if element and tag_name:
        tag = element.find(tag_name)
        return tag.text.strip() if tag else ""
    return ""

def clean_text(text):
    return text.replace('\r\n', '').replace('\xa0', '')

def scrape_kevajo_page(page_urls):
    product_data_list = []

    for page_url in page_urls:
        try:
            response = requests.get(page_url, timeout=500)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Erreur de connexion à {page_url} : {e}")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')

        # Extraction des éléments de la page
        breadcrumbs = [a.text.strip() for a in soup.select('.woocommerce-breadcrumb a')]
        
        product_title = extract_text(soup.find('h1', class_='product_title'))

        # Extraction du prix promo
        price_ins = None
        price_ins_tag = soup.find('ins', class_='woocommerce-Price-amount')
        if price_ins_tag:
            price_ins = price_ins_tag.find('bdi').text.strip()

        # Extraction du prix réel
        price_del = None
        price_del_tag = soup.find('del', class_='woocommerce-Price-amount')
        if price_del_tag:
            price_del = price_del_tag.find('bdi').text.strip()

        description_tag = soup.find('div', class_='woocommerce-product-details__short-description')
        description = extract_text(description_tag) if description_tag else ""

        attributes = {}
        attribute_rows = soup.select('.woocommerce-product-attributes tr')
        for row in attribute_rows:
            label = row.find('th').text.strip()
            value = row.find('td').text.strip()
            attributes[label] = value

        data = {
            'Breadcrumbs': breadcrumbs,
            'ProductTitle': product_title,
            'PricePromo': price_ins,
            'PriceReel': price_del,
            'Description': description,
            'Attributes': attributes,
        }
        product_data_list.append(data)

    return product_data_list

# Liste des URLs de pages produits
page_urls = list(combined_df['URL'])

# Scrape des détails de chaque page
product_data_list = scrape_kevajo_page(page_urls)

# Création d'un DataFrame à partir des données
df = pd.DataFrame(product_data_list)

# Affichage du DataFrame
df

,Breadcrumbs,ProductTitle,PricePromo,PriceReel,Description,Attributes
0,"[Accueil, Tous les produits, Eveline Cosmetics...",,None,None,,"{'Poids': '0.2 kg', 'Marque': 'Eveline Cosmeti..."
1,"[Accueil, Tous les produits, Eveline Cosmetics...",,None,None,,"{'Poids': '0.3 kg', 'Marque': 'Eveline Cosmeti..."
2,"[Accueil, Tous les produits, Eveline Cosmetics...",,None,None,,"{'Poids': '0.1 kg', 'Marque': 'Eveline Cosmeti..."
3,"[Accueil, Tous les produits, Eveline Cosmetics...",,None,None,,"{'Poids': '0.425 kg', 'Marque': 'Eveline Cosme..."
4,"[Accueil, Tous les produits, Eveline Cosmetics...",,None,None,,"{'Poids': '0.08 kg', 'Marque': 'Eveline Cosmet..."
...,...,...,...,...,...,...
109,"[Accueil, Tous les produits, Beauté et hygiène...",,None,None,,{'Poids': '0.3 kg'}
110,"[Accueil, Tous les produits, Beauté et hygiène...",,None,None,,{'Poids': '0.3 kg'}
111,"[Accueil, Tous les produits, Beauté et hygiène...",,None,None,,{'Poids': '0.3 kg'}
112,"[Accueil, Tous les produits, Beauté et hygiène...",,None,None,,{'Poids': '0.3 kg'}


# Auchan

In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://www.auchan.ci/petits-pois-moyens-400g/NJBJ4955/p?utm_source=reporting&utm_medium=spreadsheet&utm_campaign=stock-report&utm_content="

# Faire la requête HTTP
response = requests.get(url)

# Parser le contenu HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Extraire les informations nécessaires
product_title = soup.select_one('.jt-breadcrumb-title-ellipsis span:last-child').text.strip()
product_description = soup.select_one('.jt-description-content-wrapper p').text.strip()
product_price_old = soup.select_one('.jt-price-old').text.strip()
product_price_now = soup.select_one('.jt-price-now').text.strip()

# Imprimer les informations
print(f"Titre du produit: {product_title}")
print(f"Description du produit: {product_description}")
print(f"Prix d'origine: {product_price_old}")
print(f"Prix actuel: {product_price_now}")


Titre du produit: Petits pois moyens 400g
Description du produit: Les Petits Pois Moyens HARMONY de 400g sont des pois tendres et délicieux, parfaits pour agrémenter vos plats.
Prix d'origine: 750 FCFA
Prix actuel: 550 FCFA


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.auchan.ci/petits-pois-moyens-400g/NJBJ4955/p?utm_source=reporting&utm_medium=spreadsheet&utm_campaign=stock-report&utm_content="

# Faire la requête HTTP
response = requests.get(url)

# Parser le contenu HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Extraire les informations nécessaires
product_title = soup.select_one('.jt-breadcrumb-title-ellipsis span:last-child').text.strip()
product_description = soup.select_one('.jt-description-content-wrapper p').text.strip()
product_price_old = soup.select_one('.jt-price-old').text.strip()
product_price_now = soup.select_one('.jt-price-now').text.strip()

# Créer un DataFrame
data = {
    'Titre du produit': [product_title],
    'Description du produit': [product_description],
    'Prix d\'origine': [product_price_old],
    'Prix actuel': [product_price_now]
}

df = pd.DataFrame(data)

# Imprimer le DataFrame
df


,Titre du produit,Description du produit,Prix d'origine,Prix actuel
0,Petits pois moyens 400g,Les Petits Pois Moyens HARMONY de 400g sont de...,750 FCFA,550 FCFA


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Liste d'URLs
urls = [
    "https://www.auchan.ci/petits-pois-moyens-400g/NJBJ4955/p?utm_source=reporting&utm_medium=spreadsheet&utm_campaign=stock-report&utm_content="
    # Ajoutez d'autres URLs ici
]

# Initialiser une liste pour stocker les données
data_list = []

# Boucle sur les URLs
for url in urls:
    # Faire la requête HTTP
    response = requests.get(url)
    
    # Parser le contenu HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extraire les informations nécessaires
    product_title = soup.select_one('.jt-breadcrumb-title-ellipsis span:last-child').text.strip()
    product_description = soup.select_one('.jt-description-content-wrapper p').text.strip()
    product_price_old = soup.select_one('.jt-price-old').text.strip()
    product_price_now = soup.select_one('.jt-price-now').text.strip()

    # Ajouter les données à la liste
    data_list.append({
        'Titre du produit': product_title,
        'Description du produit': product_description,
        'Prix d\'origine': product_price_old,
        'Prix actuel': product_price_now
    })

# Créer un DataFrame à partir de la liste
df = pd.DataFrame(data_list)

# Imprimer le DataFrame
df


,Titre du produit,Description du produit,Prix d'origine,Prix actuel
0,Petits pois moyens 400g,Les Petits Pois Moyens HARMONY de 400g sont de...,750 FCFA,550 FCFA


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Liste d'URLs
urls = [
    "https://www.auchan.ci/petits-pois-moyens-400g/NJBJ4955/p?utm_source=reporting&utm_medium=spreadsheet&utm_campaign=stock-report&utm_content=",
    "https://www.auchan.ci/pommes-de-terre-1kg/5VORMPLL/p?utm_source=reporting&utm_medium=spreadsheet&utm_campaign=stock-report&utm_content="
    # Ajoutez d'autres URLs ici
]

# Initialiser une liste pour stocker les données
data_list = []

# Boucle sur les URLs
for url in urls:
    # Faire la requête HTTP
    response = requests.get(url)

    # Parser le contenu HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extraire les informations nécessaires avec des vérifications
    product_title_element = soup.select_one('.jt-breadcrumb-title-ellipsis span:last-child')
    product_description_element = soup.select_one('.jt-description-content-wrapper p')
    product_price_old_element = soup.select_one('.jt-price-old')
    product_price_now_element = soup.select_one('.jt-price-now')

    # Vérifier si les éléments existent avant d'accéder à leurs propriétés
    product_title = product_title_element.text.strip() if product_title_element else None
    product_description = product_description_element.text.strip() if product_description_element else None
    product_price_old = product_price_old_element.text.strip() if product_price_old_element else None
    product_price_now = product_price_now_element.text.strip() if product_price_now_element else None

    # Ajouter les données à la liste
    data_list.append({
        'Titre du produit': product_title,
        'Description du produit': product_description,
        'Prix d\'origine': product_price_old,
        'Prix actuel': product_price_now
    })

# Créer un DataFrame à partir de la liste
df = pd.DataFrame(data_list)

# Imprimer le DataFrame
df


,Titre du produit,Description du produit,Prix d'origine,Prix actuel
0,Petits pois moyens 400g,Les Petits Pois Moyens HARMONY de 400g sont de...,750 FCFA,550 FCFA
1,Pommes de terre 1kg,Une des meilleures variétés de pommes de terre...,None,750 FCFA


In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.auchan.ci/promos-can/U5DH4PLR/cp"

# Faire la requête HTTP
response = requests.get(url)

# Parser le contenu HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Extraire les informations nécessaires
product_pid = soup.select_one('.js-jt-product-card')['cy-product-pid']
product_title = soup.select_one('.js-title-line').text.strip()
product_brand = soup.select_one('.js-brand-line').text.strip()
product_image = soup.select_one('.js-image-line')['data-src']
product_price = soup.select_one('.js-price-line').text.strip()

# Ajouter les données à la liste
data_list = [{
    'Product PID': product_pid,
    'Product Title': product_title,
    'Product Brand': product_brand,
    'Product Image': product_image,
    'Product Price': product_price
}]

# Créer un DataFrame à partir de la liste
df = pd.DataFrame(data_list)

# Imprimer le DataFrame
df


,Product PID,Product Title,Product Brand,Product Image,Product Price
0,JWOY2N77,Pina colada 1L,BOUM BOUM,image:jt/p/JWOY2N77/VWKKMB7N_5414145039927.jpg,2690 FCFA


In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.auchan.ci/promos-can/U5DH4PLR/cp"

# Faire la requête HTTP
response = requests.get(url)

# Parser le contenu HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Initialiser une liste pour stocker les données
data_list = []

# Sélectionner tous les produits sur la page
products = soup.select('.js-jt-product-card')

# Boucle sur chaque produit
for product in products:
    # Extraire les informations nécessaires
    product_pid = product['cy-product-pid']
    product_title = product.select_one('.js-title-line').text.strip()
    product_brand = product.select_one('.js-brand-line').text.strip()
    product_image = product.select_one('.js-image-line')['data-src']
    product_price = product.select_one('.js-price-line').text.strip()
    product_old_price = product.select_one('.js-wasPrice-line')

    # Vérifier si le prix d'origine existe
    if product_old_price:
        product_old_price = product_old_price.text.strip()
    else:
        product_old_price = None

    # Ajouter les données à la liste
    data_list.append({
        'Product PID': product_pid,
        'Product Title': product_title,
        'Product Brand': product_brand,
        'Product Image': product_image,
        'Product Price': product_price,
        'Product Old Price': product_old_price
    })

# Créer un DataFrame à partir de la liste
df = pd.DataFrame(data_list)

# Imprimer le DataFrame
df


,Product PID,Product Title,Product Brand,Product Image,Product Price,Product Old Price
0,JWOY2N77,Pina colada 1L,BOUM BOUM,image:jt/p/JWOY2N77/VWKKMB7N_5414145039927.jpg,2690 FCFA,
1,0HLV4PBB,Rhum 18% 250ml,CALAO,image:jt/p/0HLV4PBB/0J8VGPBT_1201001.png,490 FCFA,
2,9ZFK2PBB,Rhum 70 cl,LION,image:jt/p/9ZFK2PBB/8XE9IELT_1200124.png,1525 FCFA,1750 FCFA
3,8WDO4ELL,Pur jus ananas 1L,CEBON,image:jt/p/8WDO4ELL/FZBXGVYK_6186000568123.png,1290 FCFA,
4,NWZOM955,Champagne demi sec 75cl,NICOLAS FEUILLATTE,image:jt/p/NWZOM955/LFO0GN7D_1200402.jpg,22490 FCFA,23975 FCFA
5,5ZHXIPLL,Pomme raisin 1L,PRESSEA,image:jt/p/5ZHXIPLL/NW30695K_6186000455164.png,925 FCFA,990 FCFA
6,OVWFIN77,Champagne brut 75cl,NICOLAS FEUILLATTE,image:jt/p/OVWFIN77/P7ETMB7N_1200401.jpeg,19990 FCFA,23990 FCFA
7,LO37SN77,Mini jus d'orange 200ml,PRESSEA,image:jt/p/LO37SN77/EJWQ4QWN_3d_mini_pressea_o...,170 FCFA,175 FCFA
8,7Z58APLL,Sélection 100% ananas 1L,PRESSEA,image:jt/p/7Z58APLL/38W52PLO_230213_3d_pressea...,1490 FCFA,
9,TP9JMB55,Mini jus de pomme 200ml,PRESSEA,image:jt/p/TP9JMB55/W8ZJ4P9Z_3d_mini_pressea_p...,175 FCFA,


In [12]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd



# Initialiser une liste pour stocker les données
data_list = []

# Configuration du navigateur Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
driver = webdriver.Chrome(options=options)

# Boucle sur les URLs
for url in urls:
    # Charger la page avec Selenium
    driver.get(url)

    # Attendre que la page se charge complètement
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.jt-breadcrumb-title-ellipsis span:last-child')))

    # Parser le contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extraire les informations nécessaires
    product_title = soup.select_one('.jt-breadcrumb-title-ellipsis span:last-child').text.strip()
    product_description = soup.select_one('.jt-description-content-wrapper p').text.strip()
    product_price_old = soup.select_one('.jt-price-old').text.strip()
    product_price_now = soup.select_one('.jt-price-now').text.strip()

    # Ajouter les données à la liste
    data_list.append({
        'Titre du produit': product_title,
        'Description du produit': product_description,
        'Prix d\'origine': product_price_old,
        'Prix actuel': product_price_now
    })

# Fermer le navigateur Selenium
driver.quit()

# Créer un DataFrame à partir de la liste
df = pd.DataFrame(data_list)

# Imprimer le DataFrame
df


,Titre du produit,Description du produit,Prix d'origine,Prix actuel
0,Petits pois moyens 400g,Les Petits Pois Moyens HARMONY de 400g sont de...,750 FCFA,550 FCFA


In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

# URL de la page
url = "https://www.auchan.ci/promos-can/U5DH4PLR/cp"

# Configuration du navigateur Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
driver = webdriver.Chrome(options=options)

# Charger la page avec Selenium
driver.get(url)

# Attendre que la page se charge complètement
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.js-jt-product-card')))

# Parser le contenu HTML avec BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Initialiser une liste pour stocker les données
data_list = []

# Sélectionner tous les produits sur la page
products = soup.select('.js-jt-product-card')

# Boucle sur chaque produit
for product in products:
    # Extraire les informations nécessaires
    product_pid = product['cy-product-pid']
    product_title = product.select_one('.js-title-line').text.strip()
    product_brand = product.select_one('.js-brand-line').text.strip()
    product_image = product.select_one('.js-image-line')['data-src']
    product_price = product.select_one('.js-price-line').text.strip()
    product_old_price = product.select_one('.js-wasPrice-line')

    # Vérifier si le prix d'origine existe
    if product_old_price:
        product_old_price = product_old_price.text.strip()
    else:
        product_old_price = None

    # Ajouter les données à la liste
    data_list.append({
        'Product PID': product_pid,
        'Product Title': product_title,
        'Product Brand': product_brand,
        'Product Image': product_image,
        'Product Price': product_price,
        'Product Old Price': product_old_price
    })

    # Cliquer sur l'élément pour obtenir plus de détails
    product_link = product.select_one('.js-product-anchor')
    if product_link and product_link.has_attr('href'):
        product_details_url = product_link['href']
        
        # Ouvrir un nouvel onglet pour charger la page de détails
        driver.execute_script(f"window.open('{product_details_url}', '_blank');")
        
        # Passer au nouvel onglet
        driver.switch_to.window(driver.window_handles[1])

        # Attendre que la page de détails se charge complètement
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.jt-breadcrumb-title-ellipsis span:last-child')))

        # Parser le contenu HTML de la page de détails
        detail_soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Extraire des informations supplémentaires de la page de détails
        product_description = detail_soup.select_one('.jt-description-content-wrapper p').text.strip()

        # Ajouter les informations supplémentaires à la liste
        data_list[-1]['Product Description'] = product_description

        # Fermer le nouvel onglet
        driver.close()

        # Revenir à l'onglet principal
        driver.switch_to.window(driver.window_handles[0])

# Fermer le navigateur Selenium
driver.quit()

# Créer un DataFrame à partir de la liste
df = pd.DataFrame(data_list)

# Imprimer le DataFrame
df


,Product PID,Product Title,Product Brand,Product Image,Product Price,Product Old Price,Product Description
0,JWOY2N77,Pina colada 1L,BOUM BOUM,image:jt/p/JWOY2N77/VWKKMB7N_5414145039927.jpg,2690 FCFA,,Première étape du tour du monde de notre cockt...
1,0HLV4PBB,Rhum 18% 250ml,CALAO,image:jt/p/0HLV4PBB/0J8VGPBT_1201001.png,490 FCFA,,Le rhum CALAO à 18% est idéal pour des cocktai...
2,9ZFK2PBB,Rhum 70 cl,LION,image:jt/p/9ZFK2PBB/8XE9IELT_1200124.png,1525 FCFA,1750 FCFA,Tes yeux de fauves ne peuvent pas se détourner...
3,8WDO4ELL,Pur jus ananas 1L,CEBON,image:jt/p/8WDO4ELL/FZBXGVYK_6186000568123.png,1290 FCFA,,Cêbon vous propose une gamme de jus 100% natur...
4,NWZOM955,Champagne demi sec 75cl,NICOLAS FEUILLATTE,image:jt/p/NWZOM955/LFO0GN7D_1200402.jpg,22490 FCFA,23975 FCFA,Robe or pâle avec délicatesse. Floral et fruit...
5,5ZHXIPLL,Pomme raisin 1L,PRESSEA,image:jt/p/5ZHXIPLL/NW30695K_6186000455164.png,925 FCFA,990 FCFA,Savoureuse rencontre de la pomme et du raisin....
6,OVWFIN77,Champagne brut 75cl,NICOLAS FEUILLATTE,image:jt/p/OVWFIN77/P7ETMB7N_1200401.jpeg,19990 FCFA,23990 FCFA,A l’Oeil : robe jaune pâle aux reflets turquoi...
7,LO37SN77,Mini jus d'orange 200ml,PRESSEA,image:jt/p/LO37SN77/EJWQ4QWN_3d_mini_pressea_o...,170 FCFA,175 FCFA,"Mini Présséa, la source d’énergie au quotidien..."
8,7Z58APLL,Sélection 100% ananas 1L,PRESSEA,image:jt/p/7Z58APLL/38W52PLO_230213_3d_pressea...,1490 FCFA,,"L’ananas est riche en vitamines, manganèse et ..."
9,TP9JMB55,Mini jus de pomme 200ml,PRESSEA,image:jt/p/TP9JMB55/W8ZJ4P9Z_3d_mini_pressea_p...,175 FCFA,,"Mini Présséa, la source d’énergie au quotidien..."


In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

# URL de la page
urls = ["https://www.auchan.ci/promos-can/U5DH4PLR/cp"]

# Configuration du navigateur Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
driver = webdriver.Chrome(options=options)

# Charger la page avec Selenium
for url in urls:
    driver.get(url)
    # Attendre que la page se charge complètement
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.js-jt-product-card')))
    # Parser le contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # Initialiser une liste pour stocker les données
    data_list = []
    # Sélectionner tous les produits sur la page
    products = soup.select('.js-jt-product-card')
    # Boucle sur chaque produit
    for product in products:
        # Extraire les informations nécessaires
        product_pid = product['cy-product-pid']
        product_title = product.select_one('.js-title-line').text.strip()
        product_brand = product.select_one('.js-brand-line').text.strip()
        product_image = product.select_one('.js-image-line')['data-src']
        product_price = product.select_one('.js-price-line').text.strip()
        product_old_price = product.select_one('.js-wasPrice-line')
        # Vérifier si le prix d'origine existe
        if product_old_price:
            product_old_price = product_old_price.text.strip()

        else:
            product_old_price = None

     # Ajouter les données à la liste
        data_list.append({
        'Product PID': product_pid,
        'Product Title': product_title,
        'Product Brand': product_brand,
        'Product Image': product_image,
        'Product Price': product_price,
        'Product Old Price': product_old_price
      })

# Cliquer sur l'élément pour obtenir plus de détails
        product_link = product.select_one('.js-product-anchor')
        if product_link and product_link.has_attr('href'):
            product_details_url = product_link['href']
            # Ouvrir un nouvel onglet pour charger la page de détails
            driver.execute_script(f"window.open('{product_details_url}', '_blank');")
            # Passer au nouvel onglet
            driver.switch_to.window(driver.window_handles[1])
            # Attendre que la page de détails se charge complètement
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.jt-breadcrumb-title-ellipsis span:last-child')))
            # Parser le contenu HTML de la page de détails
            detail_soup = BeautifulSoup(driver.page_source, 'html.parser')
            # Extraire des informations supplémentaires de la page de détails
            product_description = detail_soup.select_one('.jt-description-content-wrapper p').text.strip()
            # Ajouter les informations supplémentaires à la liste
            data_list[-1]['Product Description'] = product_description
            # Fermer le nouvel onglet
            driver.close()
            # Revenir à l'onglet principal
            driver.switch_to.window(driver.window_handles[0])       

# Fermer le navigateur Selenium
driver.quit()

# Créer un DataFrame à partir de la liste
df = pd.DataFrame(data_list)

# Imprimer le DataFrame
df


,Product PID,Product Title,Product Brand,Product Image,Product Price,Product Old Price,Product Description
0,JWOY2N77,Pina colada 1L,BOUM BOUM,image:jt/p/JWOY2N77/VWKKMB7N_5414145039927.jpg,2690 FCFA,,Première étape du tour du monde de notre cockt...
1,0HLV4PBB,Rhum 18% 250ml,CALAO,image:jt/p/0HLV4PBB/0J8VGPBT_1201001.png,490 FCFA,,Le rhum CALAO à 18% est idéal pour des cocktai...
2,9ZFK2PBB,Rhum 70 cl,LION,image:jt/p/9ZFK2PBB/8XE9IELT_1200124.png,1525 FCFA,1750 FCFA,Tes yeux de fauves ne peuvent pas se détourner...
3,8WDO4ELL,Pur jus ananas 1L,CEBON,image:jt/p/8WDO4ELL/FZBXGVYK_6186000568123.png,1290 FCFA,,Cêbon vous propose une gamme de jus 100% natur...
4,NWZOM955,Champagne demi sec 75cl,NICOLAS FEUILLATTE,image:jt/p/NWZOM955/LFO0GN7D_1200402.jpg,22490 FCFA,23975 FCFA,Robe or pâle avec délicatesse. Floral et fruit...
5,5ZHXIPLL,Pomme raisin 1L,PRESSEA,image:jt/p/5ZHXIPLL/NW30695K_6186000455164.png,925 FCFA,990 FCFA,Savoureuse rencontre de la pomme et du raisin....
6,OVWFIN77,Champagne brut 75cl,NICOLAS FEUILLATTE,image:jt/p/OVWFIN77/P7ETMB7N_1200401.jpeg,19990 FCFA,23990 FCFA,A l’Oeil : robe jaune pâle aux reflets turquoi...
7,LO37SN77,Mini jus d'orange 200ml,PRESSEA,image:jt/p/LO37SN77/EJWQ4QWN_3d_mini_pressea_o...,170 FCFA,175 FCFA,"Mini Présséa, la source d’énergie au quotidien..."
8,7Z58APLL,Sélection 100% ananas 1L,PRESSEA,image:jt/p/7Z58APLL/38W52PLO_230213_3d_pressea...,1490 FCFA,,"L’ananas est riche en vitamines, manganèse et ..."
9,TP9JMB55,Mini jus de pomme 200ml,PRESSEA,image:jt/p/TP9JMB55/W8ZJ4P9Z_3d_mini_pressea_p...,175 FCFA,,"Mini Présséa, la source d’énergie au quotidien..."


# Auchan

In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

# Liste d'URLs
urls = [
    "https://www.auchan.ci/promos-can/U5DH4PLR/cp",
    # Ajoutez d'autres URLs ici
]

# Configuration du navigateur Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
driver = webdriver.Chrome(options=options)

# Initialiser une liste pour stocker les données
data_list = []

# En-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Boucle sur les URLs
for url in urls:
    # Charger la page avec Selenium
    driver.get(url)

    # Attendre que la page se charge complètement
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.js-jt-product-card')))

    # Parser le contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Sélectionner tous les produits sur la page
    products = soup.select('.js-jt-product-card')

    # Boucle sur chaque produit
    for product in products:
        # Extraire les informations nécessaires
        product_pid = product['cy-product-pid']
        product_title = product.select_one('.js-title-line').text.strip()
        product_brand = product.select_one('.js-brand-line').text.strip()
        product_image = product.select_one('.js-image-line')['data-src']
        product_price = product.select_one('.js-price-line').text.strip()
        product_old_price = product.select_one('.js-wasPrice-line')

        # Vérifier si le prix d'origine existe
        if product_old_price:
            product_old_price = product_old_price.text.strip()
        else:
            product_old_price = None

        # Ajouter les données à la liste
        data_list.append({
            'Product PID': product_pid,
            'Product Title': product_title,
            'Product Brand': product_brand,
            'Product Image': product_image,
            'Product Price': product_price,
            'Product Old Price': product_old_price
        })

        # Cliquer sur l'élément pour obtenir plus de détails
        product_link = product.select_one('.js-product-anchor')
        if product_link and product_link.has_attr('href'):
            product_details_url = product_link['href']

            # Ouvrir un nouvel onglet pour charger la page de détails
            driver.execute_script(f"window.open('{product_details_url}', '_blank');")

            # Passer au nouvel onglet
            driver.switch_to.window(driver.window_handles[1])

            # Attendre que la page de détails se charge complètement
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.jt-breadcrumb-title-ellipsis span:last-child')))

            # Parser le contenu HTML de la page de détails
            detail_soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Extraire des informations supplémentaires de la page de détails
            product_description = detail_soup.select_one('.jt-description-content-wrapper p').text.strip()

            # Ajouter les informations supplémentaires à la liste
            data_list[-1]['Product Description'] = product_description

            # Fermer le nouvel onglet
            driver.close()

            # Revenir à l'onglet principal
            driver.switch_to.window(driver.window_handles[0])

# Fermer le navigateur Selenium
driver.quit()

# Créer un DataFrame à partir de la liste
df = pd.DataFrame(data_list)

# Imprimer le DataFrame
df


,Product PID,Product Title,Product Brand,Product Image,Product Price,Product Old Price,Product Description
0,JWOY2N77,Pina colada 1L,BOUM BOUM,image:jt/p/JWOY2N77/VWKKMB7N_5414145039927.jpg,2690 FCFA,,Première étape du tour du monde de notre cockt...
1,0HLV4PBB,Rhum 18% 250ml,CALAO,image:jt/p/0HLV4PBB/0J8VGPBT_1201001.png,490 FCFA,,Le rhum CALAO à 18% est idéal pour des cocktai...
2,9ZFK2PBB,Rhum 70 cl,LION,image:jt/p/9ZFK2PBB/8XE9IELT_1200124.png,1525 FCFA,1750 FCFA,Tes yeux de fauves ne peuvent pas se détourner...
3,8WDO4ELL,Pur jus ananas 1L,CEBON,image:jt/p/8WDO4ELL/FZBXGVYK_6186000568123.png,1290 FCFA,,Cêbon vous propose une gamme de jus 100% natur...
4,NWZOM955,Champagne demi sec 75cl,NICOLAS FEUILLATTE,image:jt/p/NWZOM955/LFO0GN7D_1200402.jpg,22490 FCFA,23975 FCFA,Robe or pâle avec délicatesse. Floral et fruit...
5,5ZHXIPLL,Pomme raisin 1L,PRESSEA,image:jt/p/5ZHXIPLL/NW30695K_6186000455164.png,925 FCFA,990 FCFA,Savoureuse rencontre de la pomme et du raisin....
6,OVWFIN77,Champagne brut 75cl,NICOLAS FEUILLATTE,image:jt/p/OVWFIN77/P7ETMB7N_1200401.jpeg,19990 FCFA,23990 FCFA,A l’Oeil : robe jaune pâle aux reflets turquoi...
7,7VW5GPLL,Soda orange 30cl,FANTA,image:jt/p/7VW5GPLL/KL3L2N5D_1300322.jpg,190 FCFA,225 FCFA,"Boisson rafraîchissante au jus d’orange, avec ..."
8,LO37SN77,Mini jus d'orange 200ml,PRESSEA,image:jt/p/LO37SN77/EJWQ4QWN_3d_mini_pressea_o...,170 FCFA,175 FCFA,"Mini Présséa, la source d’énergie au quotidien..."
9,7Z58APLL,Sélection 100% ananas 1L,PRESSEA,image:jt/p/7Z58APLL/38W52PLO_230213_3d_pressea...,1490 FCFA,,"L’ananas est riche en vitamines, manganèse et ..."


In [36]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

# Liste d'URLs


# Configuration du navigateur Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
driver = webdriver.Chrome(options=options)

# Initialiser une liste pour stocker les données
data_list = []

# En-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
# Boucle sur les URLs
for url in urls:
    # Charger la page avec Selenium
    driver.get(url)

    # Attendre que la page se charge complètement
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.js-jt-product-card')))

    # Parser le contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Sélectionner tous les produits sur la page
    products = soup.select('.js-jt-product-card')

    # Boucle sur chaque produit
    for product in products:
        # Extraire les informations nécessaires
        product_pid = product['cy-product-pid']
        product_title = product.select_one('.js-title-line').text.strip()
        product_brand = product.select_one('.js-brand-line').text.strip()
        product_image = product.select_one('.js-image-line')['data-src']
        product_price = product.select_one('.js-price-line').text.strip()
        product_old_price = product.select_one('.js-wasPrice-line')

        # Vérifier si le prix d'origine existe
        if product_old_price:
            product_old_price = product_old_price.text.strip()
        else:
            product_old_price = None

        # Ajouter les données à la liste
        data_list.append({
            'Product PID': product_pid,
            'Product Title': product_title,
            'Product Brand': product_brand,
            'Product Image': product_image,
            'Product Price': product_price,
            'Product Old Price': product_old_price
        })

        # Cliquer sur l'élément pour obtenir plus de détails
        product_link = product.select_one('.js-product-anchor')
        if product_link and product_link.has_attr('href'):
            product_details_url = product_link['href']

            # Ouvrir un nouvel onglet pour charger la page de détails
            driver.execute_script(f"window.open('{product_details_url}', '_blank');")

            # Passer au nouvel onglet
            driver.switch_to.window(driver.window_handles[1])

            # Attendre que la page de détails se charge complètement
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.jt-breadcrumb-title-ellipsis span:last-child')))

            # Parser le contenu HTML de la page de détails
            detail_soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Extraire des informations supplémentaires de la page de détails
            product_description = detail_soup.select_one('.jt-description-content-wrapper p').text.strip()

            # Ajouter les informations supplémentaires à la liste
            data_list[-1]['Product Description'] = product_description

            # Fermer le nouvel onglet
            driver.close()

            # Revenir à l'onglet principal
            driver.switch_to.window(driver.window_handles[0])

# Fermer le navigateur Selenium
driver.quit()

# Créer un DataFrame à partir de la liste
df = pd.DataFrame(data_list)

# Imprimer le DataFrame
df


WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome-headless-shell=120.0.6099.109)
Stacktrace:
	GetHandleVerifier [0x00007FF73E7B2142+3514994]
	(No symbol) [0x00007FF73E3D0CE2]
	(No symbol) [0x00007FF73E2776AA]
	(No symbol) [0x00007FF73E260839]
	(No symbol) [0x00007FF73E2606EB]
	(No symbol) [0x00007FF73E25FF59]
	(No symbol) [0x00007FF73E2840AD]
	(No symbol) [0x00007FF73E302824]
	(No symbol) [0x00007FF73E2E5D93]
	(No symbol) [0x00007FF73E2B4BDC]
	(No symbol) [0x00007FF73E2B5C64]
	GetHandleVerifier [0x00007FF73E7DE16B+3695259]
	GetHandleVerifier [0x00007FF73E836737+4057191]
	GetHandleVerifier [0x00007FF73E82E4E3+4023827]
	GetHandleVerifier [0x00007FF73E5004F9+689705]
	(No symbol) [0x00007FF73E3DC048]
	(No symbol) [0x00007FF73E3D8044]
	(No symbol) [0x00007FF73E3D81C9]
	(No symbol) [0x00007FF73E3C88C4]
	BaseThreadInitThunk [0x00007FF9229B257D+29]
	RtlUser

In [64]:
urls = [
    "https://www.auchan.ci/promos-can/U5DH4PLR/cp",
    "https://www.auchan.ci/mes-courses/fruits-et-legumes/CH0J2N7N",
    "https://www.auchan.ci/tout-a-moins-de-1000f/C5ESPLRU/cp",
    "https://www.auchan.ci/mes-courses/epicerie-salee/CENGQWQU",
    "https://www.auchan.ci/mes-courses/mouchoirs/CZVNGLOL",
    "https://www.auchan.ci/mes-courses/laits-en-poudre/C0PE2PBP",
    "https://www.auchan.ci/mes-courses/vinaigres-vinaigrettes-et-sauces-crudites/UZXQSLOL",
    "https://www.auchan.ci/mes-courses/bebe/U8P06ELE",
    "https://www.auchan.ci/mes-courses/huiles-epices-condiments/CQQDSP9P",
    "https://www.auchan.ci/mes-courses/pates/10EAPBPC",
    "https://www.auchan.ci/mes-courses/riz/1QH6P9PC",
    "https://www.auchan.ci/biere-cody-s-8-9-50cl-20-col/TPYPMB55/p",
    "https://www.auchan.ci/mes-courses/eaux-minerales/C7ORAPLP",
    "https://www.auchan.fr/high-tech-audio-tv-telephonie/telephonie/smartphones/ca-9531857",
    "https://www.auchan.ci/mes-courses/beurres-margarines-et-autres/UJBB6N7N",
    "https://www.auchan.ci/mes-courses/surgeles/UYB4B7B1",
    "https://www.auchan.ci/mes-courses/oeufs-beurres-et-creme/1LOLAN7N",
    "https://www.auchan.ci/nescafe-classic-200g/5XKV4PLL/p",
    "https://www.auchan.ci/mes-courses/produits-menagers-et-accessoires-de-la-maison/C3QIPLPU",
    "https://www.auchan.ci/sauce-piquante-482ml/P90HIB77/p",
    "https://www.auchan.ci/mes-courses/cremes-fraiches/CBOBGQ0Q",
    "https://www.auchan.ci/mes-courses/ustensiles-de-cuisine/1Z7JILOL",
    "https://www.auchan.ci/papyrus-papier-hygienique-parfume-decore-12x4/QO8LMP99/p",
    "https://www.auchan.ci/huile-de-carotte-100-naturelle-65ml/0XK76PBB/p",
    "https://www.auchan.ci/mixa-lait-eclat-400ml/RP0HIB77/p",
    "https://www.auchan.ci/mes-courses/dosettes-et-capsules/C3DR2PLP",
    "https://www.auchan.ci/essuie-tout-sanitex-8x3/K0L8AN55/p",
    "https://www.auchan.ci/the-instantane-citron-400g/57RT4PLL/p",
    "https://www.auchan.ci/mes-courses/wc-et-canalisations/CHHW6N7N",
    "https://www.auchan.ci/mes-courses/cereales-et-petit-dejeuner-bebe/1Y9Z6B7B",
    "https://www.auchan.ci/smiley-couche-bebe-jumbo-maxi-80-pieces/WXO56P99/p",
    "https://www.auchan.ci/savon-dermatologique-100g/ZHJKGLOO/p",
    "https://www.auchan.ci/mes-courses/colas-et-boissons-gazeuses/1NOJI959",
    "https://www.auchan.ci/mes-courses/volaille-gibier-lapin-et-autres/UZ9ZSLOL",
    "https://www.auchan.ci/mes-courses/plats-cuisines-en-conserve/1YLVIB7B",
    "https://www.auchan.ci/muesli-croustillant-500g/FRN0MVYY/p",
    "https://www.auchan.ci/cristaline-eau-minerale-33cl/WZFYSP99/p",
    "https://www.auchan.ci/eau-de-toilette-100-ml/BOF9AQ00/p",
    "https://www.auchan.ci/mes-courses/shampoings/URVNSB7B",
    "https://www.auchan.ci/22-5kg-riz-rizier-viet-lg-parf/5XVFMPLL/p",
    "https://www.auchan.ci/muesli-sans-sucre-375g/Z5TOSLOO/p",
    "https://www.auchan.ci/?trk=public_post_reshare-text",
    "https://www.auchan.ci/epicerie-salee/CBBO2Q03/cp",
    "https://www.auchan.ci/mes-courses/yaourts-et-desserts/UEX02QWQ",
    "https://www.auchan.ci/muesli-sans-sucre-375g/Z5TOSLOO/p",
    "https://www.auchan.ci/mes-courses/cremes-et-soins/1H0R6N7N",
    "https://www.auchan.ci/maman-coquillette-200g/5X8N2PLL/p",
    "https://www.auchan.ci/tablette-de-chocolat-noir-amandes-entieres-180g/QO0NIP99/p",
    "https://www.auchan.ci/fiesta-cocktail-bleu-75cl/YPWJAB77/p",
    "https://www.auchan.ci/vodka-kabisa-party-18-25cl/BFZ36Q00/p",
    "https://www.auchan.ci/mes-courses/hygiene-et-beaute/CPOSB7BU",
    "https://www.auchan.ci/caille-tradition/ZLRHGLOO/p",
    "https://www.auchan.ci/selection-boissons/1W94P9EC/cp",
    "https://www.auchan.ci/mes-courses/liqueurs-et-spiritueux/15K9GPLP",
    "https://www.auchan.ci/mes-courses/conserves-et-plats-cuisines/UBO4Q0Q1", 
    "https://www.auchan.ci/mes-courses/cremes-et-soins/1H0R6N7N",
    "https://www.auchan.ci/mes-courses/vaisselle-jetable/18LR6ELE",
    "https://www.auchan.ci/champagne-brut-75cl/OVRD2N77/p",
    "https://www.auchan.ci/mes-courses/lessives-repassage-et-soin-du-linge/1KOW2N5N",
    "https://www.auchan.ci/mes-courses-avec-livraison/1V3AB7FC/cp",
    "https://www.auchan.ci/jus-de-cassis-320ml/59FOGPLL/p",
    "https://www.auchan.ci/assaisonnement-chili-con-carne/FXVW2VYY/p",
    "https://www.auchan.ci/mes-courses-du-mois/CVYIB7FU/cp",
    "https://www.auchan.ci/savon-lessive-360g/Z8HDGLOO/p",
    "https://www.auchan.ci/couches-culottes-taille-3-x66/NFNLM955/p",
    "https://www.auchan.ci/couscous-moyen-sipa-500grs/8OH5SELL/p",
    "https://www.auchan.ci/mes-courses/ketchup-mayonnaise-moutarde-et-sauces/UO8ZSN7N",
    "https://www.auchan.ci/mes-courses/panaches/CRXB4B7B",
    "https://www.auchan.ci/savon-de-menage-175g/LXFWSN77/p",
    "https://www.auchan.ci/savon-de-toilette-beurre-de-karite-et-miel-175g/FLOJ4VYY/p",
    "https://www.auchan.ci/arachides-sales-bouteilles-75cl/0ZWOIPBB/p",
    "https://www.auchan.ci/mes-courses/gel-douche-et-bain/UEKIQWQ1",
    "https://www.auchan.ci/mes-courses/oeufs/CELZGQWQ",
    "https://www.auchan.ci/mes-courses/bles-semoules-et-cereales/UQQV6P9P",
    "https://www.auchan.ci/lait-sachet-450g/PO55SB77/p",
    "https://www.auchan.ci/whisky-70cl/BNFNMQ00/p",
    "https://www.auchan.ci/youki-moka-canette-33cl/RVNQGB77/p",
    "https://www.auchan.ci/top-lait-sachet-cafe-40g/QX3NGP99/p",
    "https://www.auchan.ci/mes-courses/couches-lingettes-et-cotons/UWO92P9P",
    "https://www.auchan.ci/mes-courses/boucherie/CW9WSP9P",
    "https://www.auchan.ci/mes-courses/plats-cuisines/1P972B7B",
    "https://www.auchan.ci/lait-concentre-sucre-1kg/WZF52P99/p",
    "https://www.auchan.ci/ananas-mini-200ml/OVTNIN77/p",
    "https://www.auchan.ci/mes-courses/biscuits-secs-sables-et-assortiments/10TMPBPC",
    "https://www.auchan.ci/mes-courses/epicerie-sucree/CDWAVYVU",
    "https://www.auchan.ci/huile-de-ricin-65-ml-20-col/JO9B6N77/p",
    "https://www.auchan.ci/mes-courses/poulet-et-coq/13QE2PLP",
    "https://www.auchan.ci/mes-courses/produits-nettoyants/1R8P4B7B",
    "https://www.auchan.ci/eau-de-javel-12-1l/9BZ06PBB/p",
    "https://www.auchan.ci/mes-courses/couches-et-couches-culottes/1ZOYSLOL",
    "https://www.auchan.ci/mes-courses/cafes/U0FDIPBP",
    "https://www.auchan.ci/mes-courses/soins-des-cheveux/UZZNGLOL",
    "https://www.auchan.ci/mes-courses/laits-et-boissons-lactees/C7NZMPLP",
    "https://www.auchan.ci/savon-375g/NPLQI955/p",
    "https://www.auchan.ci/mes-courses/serviettes-hygieniques/1EOXAQWQ",
    "https://www.auchan.ci/special-de-2000fcfa/C5ESPLRU/cp",
    "https://www.auchan.ci/200g-vermicelle-maman/3WKPMPLL/p", 
    "https://www.auchan.ci/oignon-en-poudre-50g/KX592N55/p",
    "https://www.auchan.ci/mes-courses/laits-concentres/UYEN2B7B",
    "https://www.auchan.ci/couche-9-18kg-taille-4-40-unites/FTOD2VYY/p",
    "https://www.auchan.ci/sac-pb-n-36x20-20l-dallas-20-col/QZJPGP99/p",
    "https://www.auchan.ci/mes-courses/cuisine/CTPXSB5B",
    "https://www.auchan.ci/coca-cola-33cl-canette/3XZDAPLL/p",
    "https://www.auchan.ci/mes-courses/charcuterie/13NXAPLP",
    "https://www.auchan.ci/super-clean-savon-liquide-multiusage-citron-1l/RV9P2B77/p",
    "https://www.auchan.ci/les-nouveautes/CYXMB7NU/cp",
    "https://www.auchan.ci/mes-courses/boissons/UFRY2VYV",
    "https://www.auchan.ci/mes-courses/savons/UENW2QWQ",
    "https://www.auchan.ci/mes-courses/cereales/UPNJ4B7B",
    "https://www.auchan.ci/mmm-coquille-saint-jacques-armagnac-140g/TFNKAB55/p",
    "https://www.auchan.fr/president-beurre-gastronomique-doux/pr-C1169647",
    "https://www.auchan.ci/pressea-ananas-gingembre-1l/FOF0AVYY/p",
    "https://www.auchan.ci/nectar-goyave-1l/OVYV6N77/p",
    "https://www.auchan.ci/madeleine-parfum-chocolat/0NXRMPBB/p",
    "https://www.auchan.ci/ouragan-canalisations-aux-agents-biologiques-1l/BNLF6Q00/p",
    "https://www.auchan.ci/fanico-savon-300g/BWLEIQ00/p",
    "https://www.auchan.ci/chocoquik-poudre-120g/JW7ESN77/p",
    "https://www.auchan.ci/sachet-196g/HPQL6N77/p",
    "https://www.auchan.ci/germe-de-ble-et-kasha-a-saupoudrer-180g/RYH0GB77/p",
    "https://www.auchan.ci/essuie-tout-multi-usages-maxi-jumbo/XVFPILOO/p",
    "https://www.auchan.ci/mouchoirs-africa-3-plis-240-feuilles/5BZ86PLL/p",
    "https://www.auchan.ci/clean-express-soude-caustique-500ml/DDYVAVYY/p",
    "https://www.auchan.ci/vermicelles-200g/3WKPMPLL/p",
    "https://www.auchan.ci/origan-12g/3OXJIPLL/p",
    "https://www.auchan.ci/biscuit-digestif-84g/KWT96N55/p",
    "https://www.auchan.ci/mes-courses/aperitif/1EN9IQWQ",
    "https://www.auchan.ci/mes-courses/corps/CZ02LOLU",
    "https://www.auchan.ci/mes-courses/jambons-et-saucissons-de-porc/UVZP6B7B",
    "https://www.auchan.ci/biere-33cl/LRRLIN77/p",
    "https://www.auchan.ci/savon-lessive-270g/VZ7VAB77/p",
    "https://www.auchan.ci/5l-huile-aya/0ZO0IPBB/p",
    "https://www.auchan.ci/jus-de-fraise-320ml/0E9V6PBB/p",
    "https://www.auchan.ci/javel-12-prcent-s-clean-2l/5ZKRAPLL/p",
    "https://www.auchan.ci/mes-courses/farines/1J0Y2N7N",
    "https://www.auchan.ci/lessive-nil-15g/NPL0G955/p",
    "https://www.auchan.ci/eau-de-toilette-black-ziz-100ml/JBTWIN77/p",
    "https://www.auchan.ci/mes-courses/papiers-toilette/UVWF2B7B",
    "https://www.auchan.ci/maman-vermicelle-200g/3WKPMPLL/p",
    "https://www.auchan.ci/poulet-effile/ZV8F4LOO/p",
    "https://www.auchan.ci/mes-courses/javel-et-produits-multi-usages/19XT4PBP",
    "https://www.auchan.ci/la-creme-30-20cl/ZPHZ4LOO/p",
    "https://www.auchan.ci/piments-big-sun-sent-bon-1kg/D3Y04VYY/p",
    "https://www.auchan.ci/frites-1kg/3QH5APLL/p",
    "https://www.auchan.ci/eau-de-parfum-pour-homme-100ml/EZED4QWW/p",
    "https://www.auchan.ci/yaourt-a-l-abricot-125g/3W87IPLL/p",
    "https://www.auchan.ci/mes-courses/chiens/UZZ9GLOL",
    "https://www.auchan.ci/huile-precieuse-de-ricin-65-ml/JO9B6N77/p",
    "https://www.auchan.ci/mes-courses/dentifrices-et-bains-de-bouche/19D0SPBP",
    "https://www.auchan.ci/mes-courses/brioches-et-pains-de-mie/1LQB4N7N",
    "https://www.auchan.ci/delicia-margarine-250g/0ZR96PBB/p",
    "https://www.auchan.ci/mes-courses/soins-du-visage-et-maquillage/CX99MLOL",
    "https://www.auchan.ci/linge-bebe-a-la-fleur-de-coton-x72/079T2PBB/p",
    "https://www.auchan.ci/agar-agar-x4-8g/75Y9GPLL/p",
    "https://www.auchan.ci/nescafe-breakfast-cup-32g/XVEHALOO/p",
    "https://www.auchan.ci/cuisse-de-poulet/9P3V2PBB/p",
    "https://www.auchan.ci/jus-pressea-fresh-tropic-33cl/Y7DH6B77/p",
    "https://www.auchan.ci/mes-courses/chocolat-en-poudre/1RB9SB7B",
    "https://www.auchan.ci/sac-poubelle-50l/BTOEGQ00/p",
    "https://www.auchan.ci/mes-courses/boissons-gazeuses-et-sirops/175K2PLP",
    "https://www.auchan.ci/pain-burgers-geant-brioches-x4-320g/YJVR6B77/p",
    "https://www.auchan.ci/cerelac-mais-50g/8P7J4ELL/p",
    "https://www.auchan.ci/omo-pwd-f-action-sky-12g/Y9KBMB77/p",
    "https://www.auchan.ci/dentifrice-au-charbon-100g/JBHL6N77/p",
    "https://www.auchan.ci/auchan-mayonnaise-710g/P8LKAB77/p",
    "https://www.auchan.ci/auchan-moelleux-oignons-x10-1kg/CRT5IB77/p",
    "https://www.auchan.ci/eau-de-javel-12-4l/JWPLGN77/p",
    "https://www.auchan.ci/cappuccino-280g/T8HZAB55/p",
    "https://www.auchan.ci/noix-de-coco-rapees-et-grillees-75cl/03JB2PBB/p",
    "https://www.auchan.ci/mes-courses/bieres-et-cidres/UR7QGB7B",
    "https://www.auchan.ci/mes-courses/bonbons/1RNO6B7B",
    "https://www.auchan.ci/sand-cacao-diet-natu-210gsx10/J007AN77/p",
    "https://www.auchan.ci/mes-courses/legumes-et-fruits/CRPYAB7B",
    "https://www.auchan.ci/riz-jaune-5-5kg/3KWW2PLL/p",
    "https://www.auchan.ci/mixa-lait-corps-nutritif-250ml/BWERAQ00/p",
    "https://www.auchan.ci/400g-nestle-nido-cwar/FZOB4VYY/p",
    "https://www.auchan.ci/granules-de-mil-pour-bouillie-350g/K9RJAN55/p",
    "https://www.auchan.ci/mes-courses/cafes-moulus-grains-et-chicorees/1HRRAN7N",
    "https://www.auchan.ci/coco-copaci-65ml/VPQ94B77/p",
    "https://www.auchan.ci/mes-courses/accessoires-de-cuisine-salle-de-bains-et-wc/U0DBAPBP",
    "https://www.auchan.ci/flash-citron-240g/W97PMP99/p",
    "https://www.auchan.ci/egouttoir-s-plat-blanc-12assiettes/URE2B771/p",
    "https://www.auchan.ci/mes-courses/vins/CP7F2B7B",
    "https://www.auchan.ci/cotes-de-porc-echine/KJP0IN55/p",
    "https://www.auchan.ci/gelatine-alimentaire-en-feuilles-halal-x9-17g/VYVN2B77/p",
    "https://www.auchan.ci/spled-gu10-35w-cf-2p/LXX5GN77/p",
    "https://www.auchan.ci/mes-courses/eaux/19Z32PBP",
    "https://www.auchan.ci/serviettes-hygieniques-maxi-7mm-x9/E0JOAQWW/p",
    "https://www.auchan.ci/mes-courses/fromages-rapes-et-a-cuisiner/1KJ6N5NC",
    "https://www.auchan.ci/mes-courses/thes-et-infusions/1JTX6N7N",
    "https://www.auchan.ci/auchan-lessive-gel-main/V8ZJGB77/p",
    "https://www.auchan.ci/farine-de-ble-500g/KJZLMN55/p",
    "https://www.auchan.ci/mes-courses/fromages/CV5SB7BU",
    "https://www.auchan.ci/macedoine-de-legumes-400g/0ON9MPBB/p",
    "https://www.auchan.ci/couscous-de-petit-mil-500g/DTRKIVYY/p",
    "https://www.auchan.ci/cosmia-douche-lait-fleurs-orange-250ml/1DVZ2VYY/p",
    "https://www.auchan.ci/mes-courses/tampons/C3Z06PLP",
    "https://www.auchan.ci/nutella-go-biscuits-et-pate-a-tartiner-x2-104g/FFB36VYY/p",
    "https://www.auchan.ci/selection-hygiene-beaute/CQQZ2P9E/cp",
    "https://www.auchan.ci/selection-promos/1RVIB7FC/cp",
    "https://www.auchan.ci/lessive-nil-180g/XV75ALOO/p",
    "https://www.auchan.ci/mes-courses/cuisine-et-arts-de-la-table/1X78GLOL",
    "https://www.auchan.ci/the-pur-ceylon-x100/FZJN6VYY/p",
    "https://www.auchan.ci/nescafe-classic-50g/8WPJMELL/p",
    "https://www.auchan.ci/aluminium-20m/7383IPLL/p",
    "https://www.auchan.ci/mes-courses/parapharmacie/18PEAELE",
    "https://www.auchan.ci/champagne-sans-alcool-75cl/YXTB6B77/p",
    "https://www.auchan.ci/biere-50cl/JWQP2N77/p",
    "https://www.auchan.ci/mes-courses/gel-douche-et-bain/UJ3B2N7N",
    "https://www.auchan.ci/eau-de-parfum-100ml/FHJPIVYY/p",
    "https://www.auchan.ci/beliv-aloe-vera-miel-175g/HBOQ2N77/p",
    "https://www.auchan.ci/eau-minerale-naturelle-450ml/9VWOAPBB/p",
    "https://www.auchan.ci/auchan-essuie-tout-recycle-compactuel/JJ73MN77/p",
    "https://www.auchan.ci/lait-en-poudre-demi-ecreme-400g/KWOKSN55/p",
    "https://www.auchan.ci/les-indispensables/1J56N7JC/cp",
    
    
    
    
    # Ajoutez d'autres URLs ici
]



In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from selenium.common.exceptions import TimeoutException



# Configuration du navigateur Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
#"C:\Program Files\Google\Chrome\Application\chrome.exe"
chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
driver = webdriver.Chrome(options=options)

# Initialiser une liste pour stocker les données
data_list = []

# Boucle sur les URLs
for url in urls:
    # Charger la page avec Selenium
    driver.get(url)

    try:
        # Attendre que la page se charge complètement
        wait = WebDriverWait(driver, 500)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.js-jt-product-card')))

        # Parser le contenu HTML avec BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Sélectionner tous les produits sur la page
        products = soup.select('.js-jt-product-card')

        # Boucle sur chaque produit
        for product in products:
            # Extraire les informations nécessaires
            product_pid = product['cy-product-pid']
            product_title = product.select_one('.js-title-line').text.strip()
            product_brand = product.select_one('.js-brand-line').text.strip()
            product_image = product.select_one('.js-image-line')['data-src']
            product_price = product.select_one('.js-price-line').text.strip()
            product_old_price = product.select_one('.js-wasPrice-line')

            # Vérifier si le prix d'origine existe
            if product_old_price:
                product_old_price = product_old_price.text.strip()
            else:
                product_old_price = None

            # Ajouter les données à la liste
            data_list.append({
                'Product PID': product_pid,
                'Product Title': product_title,
                'Product Brand': product_brand,
                'Product Image': product_image,
                'Product Price': product_price,
                'Product Old Price': product_old_price
            })

            # Cliquer sur l'élément pour obtenir plus de détails
            product_link = product.select_one('.js-product-anchor')
            if product_link and product_link.has_attr('href'):
                product_details_url = product_link['href']

                # Ouvrir un nouvel onglet pour charger la page de détails
                driver.execute_script(f"window.open('{product_details_url}', '_blank');")

                # Passer au nouvel onglet
                driver.switch_to.window(driver.window_handles[1])

                # Attendre que la page de détails se charge complètement
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.jt-breadcrumb-title-ellipsis span:last-child')))

                # Parser le contenu HTML de la page de détails
                detail_soup = BeautifulSoup(driver.page_source, 'html.parser')

                # Extraire des informations supplémentaires de la page de détails
                product_description = detail_soup.select_one('.jt-description-content-wrapper p').text.strip()

                # Ajouter les informations supplémentaires à la liste
                data_list[-1]['Product Description'] = product_description

                # Fermer le nouvel onglet
                driver.close()

                # Revenir à l'onglet principal
                driver.switch_to.window(driver.window_handles[0])

    except TimeoutException:
        print(f"Le délai d'attente a expiré pour l'URL : {url}")

# Fermer le navigateur Selenium
driver.quit()

# Créer un DataFrame à partir de la liste
df = pd.DataFrame(data_list)

# Imprimer le DataFrame
df


Le délai d'attente a expiré pour l'URL : https://www.auchan.ci/biere-cody-s-8-9-50cl-20-col/TPYPMB55/p
Le délai d'attente a expiré pour l'URL : https://www.auchan.fr/high-tech-audio-tv-telephonie/telephonie/smartphones/ca-9531857
Le délai d'attente a expiré pour l'URL : https://www.auchan.ci/nescafe-classic-200g/5XKV4PLL/p
Le délai d'attente a expiré pour l'URL : https://www.auchan.ci/sauce-piquante-482ml/P90HIB77/p
Le délai d'attente a expiré pour l'URL : https://www.auchan.ci/papyrus-papier-hygienique-parfume-decore-12x4/QO8LMP99/p
Le délai d'attente a expiré pour l'URL : https://www.auchan.ci/huile-de-carotte-100-naturelle-65ml/0XK76PBB/p
Le délai d'attente a expiré pour l'URL : https://www.auchan.ci/mixa-lait-eclat-400ml/RP0HIB77/p
Le délai d'attente a expiré pour l'URL : https://www.auchan.ci/essuie-tout-sanitex-8x3/K0L8AN55/p
Le délai d'attente a expiré pour l'URL : https://www.auchan.ci/the-instantane-citron-400g/57RT4PLL/p
Le délai d'attente a expiré pour l'URL : https://www.au

In [57]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import WebDriverException

# Liste des liens pour chaque catégorie
categories = [
    #"toutes les offres d'emploi",
    "administrations,moyens généraux",
    "assistanat,secrétariat",
    "comptabilite,finance",
    "metiers banque et assurances",
    "juridique,fiscal,audit,conseil",
    "RH,personnel,formation",
    "education,enseignement",
    "direction générale,direction d'unité",
    "vente,televente,assistanat",
    "commercial,technico commercial,service client",
    "responsable commercial,grands comptes",
    "créatio, design",
    "marketing, communication",
    "journalisme,médias,traduction",
    "informatique,systèmes d'information,internet",          
    "télécommunication,réseaux",
    "chantier,métiers BTP,architecture",
    "ingénierie,etudes,projet,R&D",
    "logistique,achat,stock,transport",
    "production,méthode,industrie",
    "maintenance,entretien",
    "Qualité,sécurité,Environnement",
    "Santé,Médical,Pharmacie",
    "Hotellerie,Tourisme,Restauration, Loisirs",
    "Ouvriers qualifiés, Chauffeur",
    "autre",              
    "Métiers de l'agriculture" 
]

base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]
intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# En-tête pour éviter d'être bloqué
en_tete = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialiser le pilote Selenium
# Configuration du navigateur Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
#"C:\Program Files\Google\Chrome\Application\chrome.exe"
chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
driver = webdriver.Chrome(options=options)


# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    if 'finance' in category_link:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')
    else:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(200) 
    offres = soup.find_all('div', class_='row-fluid job-details pointer')

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <a> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('a')

        # Vérifier si la balise <a> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag['href']

            try:
                # Accéder à la page de l'offre pour collecter plus d'informations
                driver.get(offre_link)
                time.sleep(200)  # Attendre un peu pour que la page se charge après le clic

                # Récupérer le contenu de la page après le clic
                soup_offre = BeautifulSoup(driver.page_source, 'html.parser')

                # Nouveau dictionnaire pour stocker les détails de l'offre
                job_details = {}

                # Extracting job details
                details_section = soup_offre.find('ul', class_='text-small')
                if details_section:
                    for li in details_section.find_all('li', class_='row-fluid'):
                        key = li.find('span', class_='span4').text.strip()
                        value = li.find('span', class_='span8').text.strip()
                        job_details[key] = value

                # Extracting the provided text
                description_section = soup_offre.find('div', class_='spaced details-description')
                if description_section:
                    provided_text = description_section.text.strip()
                    job_details['Provided Text'] = provided_text

                # Ajouter les détails de l'emploi à la liste
                all_job_details.append(job_details)

            except WebDriverException as e:
                print(f"Erreur lors de l'accès à la page de l'offre : {e}")

# Fermer le pilote Selenium à la fin
driver.quit()
# Convertir les détails des offres d'emploi en DataFrame
df_Novojob = pd.DataFrame(all_job_details)


df_Novojob


Erreur lors de l'accès à la page de l'offre : Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome-headless-shell=120.0.6099.225)
Stacktrace:
	GetHandleVerifier [0x00007FF73E7B2142+3514994]
	(No symbol) [0x00007FF73E3D0CE2]
	(No symbol) [0x00007FF73E2776AA]
	(No symbol) [0x00007FF73E26FB39]
	(No symbol) [0x00007FF73E2625DD]
	(No symbol) [0x00007FF73E2616FD]
	(No symbol) [0x00007FF73E260746]
	(No symbol) [0x00007FF73E2606EB]
	(No symbol) [0x00007FF73E25EE3D]
	(No symbol) [0x00007FF73E25F603]
	(No symbol) [0x00007FF73E27A0FD]
	(No symbol) [0x00007FF73E302E01]
	(No symbol) [0x00007FF73E2E5FEA]
	(No symbol) [0x00007FF73E3028F6]
	(No symbol) [0x00007FF73E2E5D93]
	(No symbol) [0x00007FF73E2B4BDC]
	(No symbol) [0x00007FF73E2B5C64]
	GetHandleVerifier [0x00007FF73E7DE16B+3695259]
	GetHandleVerifier [0x00007FF73E836737+4057191]
	GetHandleVerifier [0x00007FF73E82E4E3+4023827]
	GetHandleVerifier [0x00007FF73E5004F9+689705]
	(No symbol) [0x00007FF73E3DC048]
	(No symbol) [0x

ConnectTimeout: HTTPSConnectionPool(host='www.novojob.com', port=443): Max retries exceeded with url: /cote-d-ivoire/offres-d-emploi?q=assistanat+secr%C3%A9tariat (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002CAAAFF95D0>, 'Connection to www.novojob.com timed out. (connect timeout=None)'))

In [58]:
df_Novojob

,Lieu de travail,Date d'expiration,Niveau de poste,Secteur d'activité,Niveau d'étude (diplome),Nombre de postes,Type de contrat,Provided Text,Nom de l'entreprise
0,"Abidjan, Côte d'ivoire",04 Février,Manager / Responsable département,Services,"Master 1, Licence Bac + 4| Master 2, Ingénior...",01,Consultant,Notre client une société qui vient de se const...,NaN
1,"Abidjan, Côte d'ivoire",15 Février,Confirmé / Expérimenté,"Banque, Assurance, Finance","Master 2, Ingéniorat, Bac + 5",02,CDI,Missions du posteSous la supervision du Respon...,NaN
2,Côte d'ivoire,28 Février,Confirmé / Expérimenté,Industries,"Master 2, Ingéniorat, Bac + 5",01 poste ouvert,CDI| CDD,ENTITE : ...,Société Générale Côte D'ivoire
3,"Abidjan, Côte d'ivoire",21 Février,Confirmé / Expérimenté,"Banque, Assurance, Finance","Master 2, Ingéniorat, Bac + 5",01,CDI,AVIS DE RECRUTEMENT FAMILLE D'EMPLOIS : DEVELO...,NaN
4,"Abidjan, Côte d'ivoire",21 Janvier,Confirmé / Expérimenté,"Banque, Assurance, Finance","Master 1, Licence Bac + 4| Master 2, Ingénior...",01,CDI| CDD,La banque en toute confiance. Acteur majeur du...,NaN
5,"Abidjan, Côte d'ivoire",04 Février,Responsable d'équipe| Confirmé / Expérimenté,Services,"Master 1, Licence Bac + 4",01,CDI| CDD,Le TECH BAR est un coworking qui permettra aux...,NaN
6,"Abidjan, Côte d'ivoire",04 Février,Responsable d'équipe| Confirmé / Expérimenté,Services,"Master 1, Licence Bac + 4",01,CDI| CDD,Le CENTRE FINANCIER est une unité au sein de l...,NaN
7,"Abidjan, Côte d'ivoire",04 Février,Responsable d'équipe| Confirmé / Expérimenté,Services,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",01,CDI| CDD,Le FOOD COURT est un restaurant qui permettra ...,NaN
8,Côte d'ivoire,28 Février,Responsable d'équipe| Confirmé / Expérimenté,Industries,"Master 2, Ingéniorat, Bac + 5",01 poste ouvert,CDI| CDD,ENTITE : ...,Société Générale Côte D'ivoire
9,Côte d'ivoire,28 Janvier,Confirmé / Expérimenté,Industries,"Master 1, Licence Bac + 4| Master 2, Ingénior...",02 postes ouverts,CDD,ENTITE : Sociét...,Société Générale Côte D'ivoire


In [59]:
df_offers = pd.DataFrame({
    'Intitule': intitules_list,
    'Entreprise': entreprises_list,
    'Pays': pays_list,
    'Date': dates_list,
    'Niveau': niveau_list,
    'Experience_lettre': experience_list,
    'Lien': lien_list
})
df_offers

,Intitule,Entreprise,Pays,Date,Niveau,Experience_lettre,Lien
0,Consultant Manager Général,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Manager / Responsable département,6 à 10 ans,0
1,Administrateur Systèmes et Réseaux,Société Ivoirienne de Banque (SIB),"Abidjan, Côte d'ivoire",03 Janvier,Confirmé / Expérimenté,3 à 5 ans,0
2,HRBP Senior – H/F,Société Générale Côte D'ivoire,Côte d'ivoire,30 Novembre 2023,Confirmé / Expérimenté,6 à 10 ans,0
3,Directeur d'Agence (h/f),BICICI,"Abidjan, Côte d'ivoire",23 Novembre 2023,Confirmé / Expérimenté,3 à 5 ans,0
4,Chargé de Prévention et Sécurité au Travail (...,Entreprise anonyme,"Abidjan, Côte d'ivoire",23 Octobre 2023,Confirmé / Expérimenté,3 à 5 ans,0
5,Gestionnaire Tech Bar,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Responsable d'équipe,6 à 10 ans,0
6,Gestionnaire Centre Financier,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Responsable d'équipe,6 à 10 ans,0
7,Gestionnaire Food Court,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Responsable d'équipe,6 à 10 ans,0
8,Superviseur Maintenance Bâtiments - H/F,Société Générale Côte D'ivoire,Côte d'ivoire,30 Novembre 2023,Responsable d'équipe,6 à 10 ans,0
9,Chef de Projet ( Gestion Applicative & Projets...,Société Générale Côte D'ivoire,Côte d'ivoire,30 Octobre 2023,Confirmé / Expérimenté,6 à 10 ans,0


In [60]:
df_Novojob = pd.DataFrame(all_job_details)
df_Novojob

,Lieu de travail,Date d'expiration,Niveau de poste,Secteur d'activité,Niveau d'étude (diplome),Nombre de postes,Type de contrat,Provided Text,Nom de l'entreprise
0,"Abidjan, Côte d'ivoire",04 Février,Manager / Responsable département,Services,"Master 1, Licence Bac + 4| Master 2, Ingénior...",01,Consultant,Notre client une société qui vient de se const...,NaN
1,"Abidjan, Côte d'ivoire",15 Février,Confirmé / Expérimenté,"Banque, Assurance, Finance","Master 2, Ingéniorat, Bac + 5",02,CDI,Missions du posteSous la supervision du Respon...,NaN
2,Côte d'ivoire,28 Février,Confirmé / Expérimenté,Industries,"Master 2, Ingéniorat, Bac + 5",01 poste ouvert,CDI| CDD,ENTITE : ...,Société Générale Côte D'ivoire
3,"Abidjan, Côte d'ivoire",21 Février,Confirmé / Expérimenté,"Banque, Assurance, Finance","Master 2, Ingéniorat, Bac + 5",01,CDI,AVIS DE RECRUTEMENT FAMILLE D'EMPLOIS : DEVELO...,NaN
4,"Abidjan, Côte d'ivoire",21 Janvier,Confirmé / Expérimenté,"Banque, Assurance, Finance","Master 1, Licence Bac + 4| Master 2, Ingénior...",01,CDI| CDD,La banque en toute confiance. Acteur majeur du...,NaN
5,"Abidjan, Côte d'ivoire",04 Février,Responsable d'équipe| Confirmé / Expérimenté,Services,"Master 1, Licence Bac + 4",01,CDI| CDD,Le TECH BAR est un coworking qui permettra aux...,NaN
6,"Abidjan, Côte d'ivoire",04 Février,Responsable d'équipe| Confirmé / Expérimenté,Services,"Master 1, Licence Bac + 4",01,CDI| CDD,Le CENTRE FINANCIER est une unité au sein de l...,NaN
7,"Abidjan, Côte d'ivoire",04 Février,Responsable d'équipe| Confirmé / Expérimenté,Services,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",01,CDI| CDD,Le FOOD COURT est un restaurant qui permettra ...,NaN
8,Côte d'ivoire,28 Février,Responsable d'équipe| Confirmé / Expérimenté,Industries,"Master 2, Ingéniorat, Bac + 5",01 poste ouvert,CDI| CDD,ENTITE : ...,Société Générale Côte D'ivoire
9,Côte d'ivoire,28 Janvier,Confirmé / Expérimenté,Industries,"Master 1, Licence Bac + 4| Master 2, Ingénior...",02 postes ouverts,CDD,ENTITE : Sociét...,Société Générale Côte D'ivoire


In [34]:
df_offers = pd.DataFrame({
    'Intitule': intitules_list,
    'Entreprise': entreprises_list,
    'Pays': pays_list,
    'Date': dates_list,
    'Niveau': niveau_list,
    'Experience_lettre': experience_list,
    'Lien': lien_list
})
df_offers

,Intitule,Entreprise,Pays,Date,Niveau,Experience_lettre,Lien
0,Consultant Manager Général,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Manager / Responsable département,6 à 10 ans,0
1,Administrateur Systèmes et Réseaux,Société Ivoirienne de Banque (SIB),"Abidjan, Côte d'ivoire",03 Janvier,Confirmé / Expérimenté,3 à 5 ans,0
2,Acheteur - H/F,Société Générale Côte D'ivoire,Côte d'ivoire,18 Octobre 2023,Confirmé / Expérimenté,6 à 10 ans,0
3,HRBP Senior – H/F,Société Générale Côte D'ivoire,Côte d'ivoire,30 Novembre 2023,Confirmé / Expérimenté,6 à 10 ans,0
4,Directeur d'Agence (h/f),BICICI,"Abidjan, Côte d'ivoire",23 Novembre 2023,Confirmé / Expérimenté,3 à 5 ans,0
...,...,...,...,...,...,...,...
610,Stagiaire Gestion de Projets Digitaux,Société Générale Afrique de l'ouest ( Centre ...,"Abidjan, Côte d'ivoire",29 Novembre 2023,Débutant / Junior,Moins d’un an,25
611,Auditeur Senior des Systèmes d'Information,Société Générale Afrique de l'ouest ( Centre ...,"Abidjan, Côte d'ivoire",09 Novembre 2023,Responsable d'équipe,3 à 5 ans,25
612,Technicien de Traitement des Eaux (H/F),SOLIBRA (Société de Limonaderies et de Brasser...,"Abidjan, Côte d'ivoire",10 Janvier,Confirmé / Expérimenté,1 à 2 ans,25
613,Chargé de Support Système Windows et Virtualis...,Ascens,"Abidjan, Côte d'ivoire",24 Novembre 2023,Confirmé / Expérimenté,6 à 10 ans,25


In [ ]:
# Ajouter les listes existantes en tant que colonnes au DataFrame
df_Novojob['Intitule'] = intitules_list
df_Novojob['Entreprise'] = entreprises_list
df_Novojob['Pays'] = pays_list
df_Novojob['Date'] = dates_list
df_Novojob['Niveau'] = niveau_list
df_Novojob['Experience_lettre'] = experience_list
df_Novojob['Lien'] = lien_list

# Réorganiser les colonnes selon vos besoins
df_Novojob = df_Novojob[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien', 'Lieu de travail', "Date d'expiration", 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplome)",
    "Nombre de postes", "Type de contrat", "Provided Text", "Nom de l'entreprise"
]]

# Afficher le DataFrame